In [1]:
%run 01_Model_CFE.ipynb


c:\Users\Emil\anaconda3\envs\Backup_PY-Opt_I_Hope\lib\site-packages\nbformat\__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


ADS: frdp_c --> AMOR + ppi_c
CPR1_CYP71AV1: AMOR + h_c + nadph_c + o2_c --> AAOH + h2o_c + nadp_c
ADH1: AAOH + nad_c --> AAld + h_c + nadh_c
ALDH1: AAld + h2o_c + nadp_c --> AA + h_c + nadph_c


In [2]:
import numpy as np
from tqdm import tqdm

from scipy.integrate import solve_ivp

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import cobra


In [4]:
def add_dynamic_bounds(iMM904, y):
    """Use external concentrations to bound the uptake flux of sucrose."""
    biomass, sucrose, oxygen = y  # expand the boundary species
    sucrose_max_import = -10 * sucrose / (5 + sucrose)
   

    iMM904.reactions.EX_sucr_e.lower_bound = sucrose_max_import


def dynamic_system(t, y):
    """Calculate the time derivative of external species."""

    biomass, sucrose, oxygen = y  # expand the boundary species

    # Calculate the specific exchanges fluxes at the given external concentrations.
    with iMM904:
        add_dynamic_bounds(iMM904, y)

        cobra.util.add_lp_feasibility(iMM904)
        feasibility = cobra.util.fix_objective_as_constraint(iMM904)
        lex_constraints = cobra.util.add_lexicographic_constraints(
            iMM904, ['BIOMASS_SC5_notrace', 'EX_sucr_e', 'EX_o2_e'], ['max', 'max', 'max'])
    fluxes = lex_constraints.values
    fluxes *= biomass
    if dynamic_system.pbar is not None:
        dynamic_system.pbar.update(1)
        dynamic_system.pbar.set_description('t = {:.3f}'.format(t))

    return fluxes


In [5]:
dynamic_system.pbar = None

In [6]:

def infeasible_event(t, y):
    with iMM904:

        add_dynamic_bounds(iMM904, y)

        cobra.util.add_lp_feasibility(iMM904)
        feasibility = cobra.util.fix_objective_as_constraint(iMM904)

    return feasibility - infeasible_event.epsilon

infeasible_event.epsilon = 1E-6
infeasible_event.direction = 1
infeasible_event.terminal = True

In [7]:
ts = np.linspace(0, 15, 100)  # Desired integration resolution and interval
y0 = [0.1, 10,20]

with tqdm() as pbar:
    dynamic_system.pbar = pbar

    sol = solve_ivp(
        fun=dynamic_system,
        events=[infeasible_event],
        t_span=(ts.min(), ts.max()),
        y0=y0,
        t_eval=ts,
        rtol=1e-6,
        atol=1e-8,
        method='BDF'
    )

    

t = 7.798: : 329it [10:55,  1.99s/it]


Infeasible: None (infeasible).

In [ ]:
sol

In [ ]:
ax = plt.subplot(111)
ax.plot(sol.t, sol.y.T[:, 0])
ax2 = plt.twinx(ax)
ax2.plot(sol.t, sol.y.T[:, 1], color='r')
ax3 = plt.twinx(ax2)
ax3.plot(sol.t, sol.y.T[:, 2], color='g')

ax.set_ylabel('Biomass', color='b')
ax2.set_ylabel('Sucrose', color='r')
ax3.set_ylabel('test', color='g')